[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dakimura/jquants-sample/blob/main/data_retrieve.ipynb)

# J-Quantsチュートリアル用データ取得

本ノートブックでは、J-Quantsチュートリアルセミナーに先立って
セミナーで必要となる株価・財務情報データをJ-Quants APIを使用して取得し、
ローカルホストに保存するところまでを行います。
本ノートブックの動作確認環境はpyproject.tomlを参照ください。
TODO: Google Colab上での動作確認


## J-Quants API
J-Quants APIにアクセスするためには [J-Quants API の Web サイト](https://application.jpx-jquants.com/) から取得できるリフレッシュトークンが必要になります。
会員登録とリフレッシュトークンの取得を完了させてから次に進みます。

必要なライブラリのinstall/importと設定を行います。

In [1]:
!pip install numpy jquants-api-client==0.1.2

import getpass
from requests import HTTPError
from datetime import datetime

import jquantsapi
import pandas as pd
import numpy as np
from dateutil import tz

# --- コンフィグ ---
# データを保存するGoogleドライブ上のディレクトリ
# STORAGE_DIR_PATH = "/content/drive/MyDrive/drive_ws/marketdata"
# デバッグ中
STORAGE_DIR_PATH = "/tmp/marketdata"
# CSVデータを保存するファイルパス
stock_fins_csvfile_path = STORAGE_DIR_PATH + "/stock_fin_debug.csv"
stock_price_csvfile_path = STORAGE_DIR_PATH + "/stock_price_debug.csv"

# J-Quants API から取得するデータの期間
#start_dt: datetime = datetime(2017, 1, 1, tzinfo=tz.gettz("Asia/Tokyo"))
# デバッグ中
start_dt: datetime = datetime(2022, 7, 24, tzinfo=tz.gettz("Asia/Tokyo"))

end_dt: datetime = datetime(2022, 7, 31, tzinfo=tz.gettz("Asia/Tokyo"))

You should consider upgrading via the '/Users/dakimura/Library/Caches/pypoetry/virtualenvs/jquants-sample-WjxObdxd-py3.9/bin/python -m pip install --upgrade pip' command.


https://application.jpx-jquants.com/ から取得したJ-Quants APIのリフレッシュトークンを入力してください。
TODO: リフレッシュトークン取得APIに対応する。ID/Passwordは環境変数にでも設定してもらうようにする？

In [2]:
refresh_token = getpass.getpass(prompt="J-Quants Refresh Token:")
# リフレッシュトークンを使用できるか検証します。
test_cli = jquantsapi.Client(refresh_token=refresh_token)
try:
    id_token = test_cli.get_id_token()
    if len(id_token) > 0:
        print("refresh_tokenは正常です。次の手順に進んでください。")
except HTTPError:
    print("refresh_tokenを使用できません。再度値を確認してください。")

refresh_tokenは正常です。次の手順に進んでください。


データ取得期間(start_dt <= 時刻 <= end_dt) の全銘柄の価格と財務情報データを取得します。
このステップの実行には環境によって数十分ほど時間がかかります。

In [14]:
cli = jquantsapi.Client(refresh_token=refresh_token)
stock_price: pd.DataFrame = cli.get_price_range(start_dt=start_dt, end_dt=end_dt)
stock_fin: pd.DataFrame = cli.get_statements_range(start_dt=start_dt, end_dt=end_dt)

取得したデータをCSV形式で保存します。

In [40]:
pd.to_csv(stock_price, stock_price_csvfile_path)
pd.to_csv(stock_fin, stock_fins_csvfile_path)

このnotebookはデータ保存までで終了です。predictor.ipynbに進みましょう。